In [11]:
import pandas as pd
import plotly.express as px
import json

In [12]:
data = pd.read_csv('data/time_series_covid19_confirmed_global_iso3_regions.csv')

In [13]:
data.drop(index=0, inplace=True)

In [14]:
data.columns[:4]

Index(['Province/State', 'Country/Region', 'Lat', 'Long'], dtype='object')

In [15]:
data.columns[-7:]

Index(['ISO 3166-1 Alpha 3-Codes', 'Region Code', 'Region Name',
       'Sub-region Code', 'Sub-region Name', 'Intermediate Region Code',
       'Intermediate Region Name'],
      dtype='object')

In [16]:
regions = data.drop(columns=data.columns[:4]).drop(columns=['ISO 3166-1 Alpha 3-Codes', 'Region Code','Sub-region Code', 'Sub-region Name', 'Intermediate Region Code','Intermediate Region Name']).groupby('Region Name').sum().T

In [17]:
regions.columns.name=None

In [18]:
regions.index = pd.to_datetime(regions.index, format='%m/%d/%y') 

In [19]:
regions.to_csv('data_ml/regions.csv')

In [ ]:
px.line(regions.diff().applymap(lambda x : max(0,x)), y=regions.columns, height=800)

In [ ]:
countries = data.set_index(['ISO 3166-1 Alpha 3-Codes',"Country/Region"]).drop(columns=['Province/State','Lat', 'Long']).drop(columns=['Region Name', 'Region Code','Sub-region Code', 'Sub-region Name', 'Intermediate Region Code','Intermediate Region Name']).T

In [ ]:
countries.index = pd.to_datetime(countries.index, format='%m/%d/%y') 

In [ ]:
countries

In [ ]:
countries = countries.resample('Q').sum().reset_index().melt(id_vars=['index'])

In [ ]:
countries

In [ ]:
countries.columns = ['Date','ISO_A3','Country/Region','Cases']

In [ ]:
countries

In [ ]:
GeoJson = json.load(open("data/countries.geojson", "r"))

In [ ]:
fig = px.choropleth_mapbox(
    countries,
    geojson=GeoJson,
    locations='ISO_A3',
    color='Cases',
    featureidkey="properties.ISO_A3",
    zoom=1,
    mapbox_style="open-street-map",
    width=1200,
    height=900,
    opacity=0.5,
    animation_frame='Date',
    color_continuous_scale="amp",
    range_color=(0,3000000000)
    
)

In [ ]:
fig.write_html('Maps_20_23.html')